In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
from keras import regularizers
from keras.regularizers import l2
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, Lambda, Dropout
from keras.layers import SeparableConv2D, Add, Convolution2D, concatenate, Layer, ReLU, DepthwiseConv2D, Reshape, Multiply, InputSpec
from keras.models import Model, load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
from PIL import Image
from tqdm import tqdm, trange
import random
from keras.applications import Xception, VGG16
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [2]:
nb_classes = 2  # number of classes
img_width, img_height = 64, 64  # change based on the shape/structure of your images
batch_size = 128  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 300  # number of iteration the algorithm gets trained.

In [3]:
train_dir = '/home/www/fake_detection/task2/128x128/train'
validation_dir = '/home/www/fake_detection/task2/128x128/validation'
test50_dir = '/home/www/fake_detection/task2/128x128/test_50'

In [4]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"
# Modular function for Fire Node

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x


# Original SqueezeNet from paper.



img_input = Input(shape=(img_height, img_width, 3))

x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
x = Activation('relu', name='relu_conv1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

x = fire_module(x, fire_id=2, squeeze=16, expand=64)
x = fire_module(x, fire_id=3, squeeze=16, expand=64)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

x = fire_module(x, fire_id=4, squeeze=32, expand=128)
x = fire_module(x, fire_id=5, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

x = fire_module(x, fire_id=6, squeeze=48, expand=192)
x = fire_module(x, fire_id=7, squeeze=48, expand=192)
x = fire_module(x, fire_id=8, squeeze=64, expand=256)
x = fire_module(x, fire_id=9, squeeze=64, expand=256)

# x_dp = Dropout(0.5, name='drop9')(x)
x_conv = Convolution2D(nb_classes, (1, 1), padding='valid', name='conv10')(x)
x_relu = Activation('relu', name='relu_conv10')(x_conv)
x_gap = GlobalAveragePooling2D()(x_relu)
x_sm = Activation('softmax', name='loss')(x_gap)

model = Model(img_input, x_sm, name='squeezenet')
model.summary()


Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 31, 31, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 31, 31, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 15, 15, 64)   0           relu_conv1[0][0]                 
________________________________________________________________________________________

In [5]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(len(model.trainable_weights))

52


In [6]:
train_datagen = ImageDataGenerator(rotation_range=0.0, 
                                   shear_range=0.0,
                                   zoom_range=0.0,
                                   width_shift_range=0.0,
                                   height_shift_range=0.0,
                                   horizontal_flip=False,
                                   rescale=1./255,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        class_mode='categorical')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')

Found 128404 images belonging to 2 classes.
Found 32100 images belonging to 2 classes.
Found 37566 images belonging to 2 classes.


In [7]:
# callback_list = [EarlyStopping(monitor='val_accuracy', patience=10),
#                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)]
# history = model.fit_generator(train_generator,
#                             steps_per_epoch=200,
#                             epochs=100,
#                             validation_data=validation_generator,
#                             validation_steps=len(validation_generator),
#                             callbacks=callback_list)

In [8]:
# model.save('/home/www/fake_detection/model/celeba_squeezenet.h5')

In [9]:
# model = load_model('/home/www/fake_detection/model/celeba_squeezenet.h5')

In [10]:
# output = model.predict_generator(test50_generator, steps=len(test50_generator), verbose=1)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# print(test50_generator.class_indices)
# print(output)

294/294 [==============================] - 28s 95ms/step
{'0_real': 0, '1_fake': 1}
[[0.500 0.500]
 [0.500 0.500]
 [0.500 0.500]
 ...
 [0.500 0.500]
 [0.500 0.500]
 [0.500 0.500]]


In [11]:
# output_score50 = []
# output_class50 = []
# answer_class50 = []
# answer_class50_1 =[]

# for i in trange(len(test50_generator)):
#     output50 = model.predict_on_batch(test50_generator[i][0])
#     output_score50.append(output50)
#     answer_class50.append(test50_generator[i][1])
    
# output_score50 = np.concatenate(output_score50)
# answer_class50 = np.concatenate(answer_class50)

# output_class50 = np.argmax(output_score50, axis=1)
# answer_class50_1 = np.argmax(answer_class50, axis=1)

# print(output_class50)
# print(answer_class50_1)

100%|██████████| 294/294 [00:52<00:00,  5.56it/s]

[0 0 0 ... 0 0 0]
[0 0 0 ... 1 1 1]


In [12]:
# cm50 = confusion_matrix(answer_class50_1, output_class50)
# report50 = classification_report(answer_class50_1, output_class50)

# recall50 = cm50[0][0] / (cm50[0][0] + cm50[0][1])
# fallout50 = cm50[1][0] / (cm50[1][0] + cm50[1][1])

# fpr50, tpr50, thresholds50 = roc_curve(answer_class50_1, output_score50[:, 1], pos_label=1.)
# eer50 = brentq(lambda x : 1. - x - interp1d(fpr50, tpr50)(x), 0., 1.)
# thresh50 = interp1d(fpr50, thresholds50)(eer50)

# print(report50)
# print(cm50)
# print("AUROC: %f" %(roc_auc_score(answer_class50_1, output_score50[:, 1])))
# print(thresh50)
# print('test_acc: ', len(output_class50[np.equal(output_class50, answer_class50_1)]) / len(output_class50))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67     18788
           1       0.00      0.00      0.00     18778

    accuracy                           0.50     37566
   macro avg       0.25      0.50      0.33     37566
weighted avg       0.25      0.50      0.33     37566

[[18788     0]
 [18778     0]]
AUROC: 0.500000
1.0
test_acc:  0.5001330990789544


/home/www/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
def cutout(img):
    """
    # Function: RandomCrop (ZeroPadded (4, 4)) + random occulusion image
    # Arguments:
        img: image
    # Returns:
        img
    """
    height = img.shape[0]
    width = img.shape[1]
    channels = img.shape[2]
    MAX_CUTS = 3 # chance to get more cuts
    MAX_LENGTH_MUTIPLIER = 5 # chance to get larger cuts
    # 16 for cifar10, 8 for cifar100
    
    # Zero-padded (4, 4)
#     img = np.pad(img, ((4,4),(4,4),(0,0)), mode='constant', constant_values=(0))
    
#     # random-crop 64x64
#     dy, dx = height, width
#     x = np.random.randint(0, width - dx + 1)
#     y = np.random.randint(0, height - dy + 1)
#     img = img[y:(y+dy), x:(x+dx)]
    
#     mean norm
#     mean = img.mean(keepdims=True)
#     img -= mean

    img *= 1./255
    
    mask = np.ones((height, width, channels), dtype=np.float32)
    nb_cuts = np.random.randint(0, MAX_CUTS + 1)
    
    # cutout
    for i in range(nb_cuts):
        y = np.random.randint(height)
        x = np.random.randint(width)
        length = 4 * np.random.randint(1, MAX_LENGTH_MUTIPLIER+1)
        
        y1 = np.clip(y-length//2, 0, height)
        y2 = np.clip(y+length//2, 0, height)
        x1 = np.clip(x-length//2, 0, width)
        x2 = np.clip(x+length//2, 0, width)
        
        mask[y1:y2, x1:x2, :] = 0.
    
    img = img * mask
    return img

In [14]:
class ReLU6(Layer):
    def __init__(self):
        super().__init__(name="ReLU6")
        self.relu6 = ReLU(max_value=6, name="ReLU6")

    def call(self, input):
        return self.relu6(input)


class HardSigmoid(Layer):
    def __init__(self):
        super().__init__()
        self.relu6 = ReLU6()

    def call(self, input):
        return self.relu6(input + 3.0) / 6.0


class HardSwish(Layer):
    def __init__(self):
        super().__init__()
        self.hard_sigmoid = HardSigmoid()

    def call(self, input):
        return input * self.hard_sigmoid(input)
    
class Attention(Layer):
    def __init__(self, ch, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.channels = ch
        self.filters_f_g = self.channels // 8
        self.filters_h = self.channels

    def build(self, input_shape):
        kernel_shape_f_g = (1, 1) + (self.channels, self.filters_f_g)
        print(kernel_shape_f_g)
        kernel_shape_h = (1, 1) + (self.channels, self.filters_h)

        # Create a trainable weight variable for this layer:
        self.gamma = self.add_weight(name='gamma', shape=[1], initializer='zeros', trainable=True)
        self.kernel_f = self.add_weight(shape=kernel_shape_f_g,
                                        initializer='glorot_uniform',
                                        name='kernel_f')
        self.kernel_g = self.add_weight(shape=kernel_shape_f_g,
                                        initializer='glorot_uniform',
                                        name='kernel_g')
        self.kernel_h = self.add_weight(shape=kernel_shape_h,
                                        initializer='glorot_uniform',
                                        name='kernel_h')
        self.bias_f = self.add_weight(shape=(self.filters_f_g,),
                                      initializer='zeros',
                                      name='bias_F')
        self.bias_g = self.add_weight(shape=(self.filters_f_g,),
                                      initializer='zeros',
                                      name='bias_g')
        self.bias_h = self.add_weight(shape=(self.filters_h,),
                                      initializer='zeros',
                                      name='bias_h')
        super(Attention, self).build(input_shape)
        # Set input spec.
        self.input_spec = InputSpec(ndim=4,
                                    axes={3: input_shape[-1]})
        self.built = True


    def call(self, x):
        def hw_flatten(x):
            return K.reshape(x, shape=[K.shape(x)[0], K.shape(x)[1]*K.shape(x)[2], K.shape(x)[-1]])

        f = K.conv2d(x,
                     kernel=self.kernel_f,
                     strides=(1, 1), padding='same')  # [bs, h, w, c']
        f = K.bias_add(f, self.bias_f)
        g = K.conv2d(x,
                     kernel=self.kernel_g,
                     strides=(1, 1), padding='same')  # [bs, h, w, c']
        g = K.bias_add(g, self.bias_g)
        h = K.conv2d(x,
                     kernel=self.kernel_h,
                     strides=(1, 1), padding='same')  # [bs, h, w, c]
        h = K.bias_add(h, self.bias_h)

        s = tf.matmul(hw_flatten(g), hw_flatten(f), transpose_b=True)  # # [bs, N, N]

        beta = K.softmax(s, axis=-1)  # attention map

        o = K.batch_dot(beta, hw_flatten(h))  # [bs, N, C]

        o = K.reshape(o, shape=K.shape(x))  # [bs, h, w, C]
        x = self.gamma * o + x

        return x

    def compute_output_shape(self, input_shape):
        return input_shape

In [15]:
ft_dir = '/home/www/fake_detection/task2/128x128/fine_tune'
train_gen_aug = ImageDataGenerator(shear_range=0, 
                               zoom_range=0.2,
                               rotation_range=0.2,
                               width_shift_range=2., 
                               height_shift_range=2.,
                               horizontal_flip=True,
                               zca_whitening=False,
                               fill_mode='nearest',
                               preprocessing_function=cutout)
test_datagen = ImageDataGenerator(rescale=1./255)

ft_gen = train_gen_aug.flow_from_directory(ft_dir,
                                              target_size=(img_height, img_width),
                                              batch_size=batch_size,
                                              shuffle=True,
                                              class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        class_mode='categorical')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 32100 images belonging to 2 classes.
Found 37566 images belonging to 2 classes.


In [16]:
model_ft = load_model('/home/www/fake_detection/model/celeba_squeezenet.h5')
for i in range(4):
    model_ft.layers.pop()
im_in = Input(shape=(img_width, img_height, 3))

base_model = Model(img_input, x)
base_model.set_weights(model_ft.get_weights())
for i in range(len(base_model.layers) - 0):
    base_model.layers[i].trainable = False
    
x1 = base_model(im_in) # (12, 12, 32)
########### Mobilenet block bneck 3x3 (32 --> 128) #################
expand1 = Conv2D(576, kernel_size=1, strides=1, kernel_regularizer=l2(1e-5), use_bias=False)(x1)
expand1 = BatchNormalization()(expand1)
expand1 = HardSwish()(expand1)
dw1 = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), use_bias=False)(expand1)
dw1 = BatchNormalization()(dw1)
se_gap1 = GlobalAveragePooling2D()(dw1)
se_gap1 = Reshape([1, 1, -1])(se_gap1)
se1 = Conv2D(144, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se_gap1)
se1 = Activation('relu')(se1)
se1 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se1)
se1 = HardSigmoid()(se1)
se1 = Multiply()([expand1, se1])
project1 = HardSwish()(se1)
project1 = Conv2D(128, kernel_size=(1, 1), padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project1)
project1 = BatchNormalization()(project1)

########### Mobilenet block bneck 5x5 (128 --> 128) #################
expand2 = Conv2D(576, kernel_size=1, strides=1, kernel_regularizer=l2(1e-5), use_bias=False)(project1)
expand2 = BatchNormalization()(expand2)
expand2 = HardSwish()(expand2)
dw2 = DepthwiseConv2D(kernel_size=(5,5), strides=(1,1), padding='same', depthwise_regularizer=l2(1e-5), use_bias=False)(expand2)
dw2 = BatchNormalization()(dw2)
se_gap2 = GlobalAveragePooling2D()(dw2)
se_gap2 = Reshape([1, 1, -1])(se_gap2)
se2 = Conv2D(144, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se_gap2)
se2 = Activation('relu')(se2)
se2 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se2)
se2 = HardSigmoid()(se2)
se2 = Multiply()([expand2, se2])
project2 = HardSwish()(se2)
project2 = Conv2D(128, kernel_size=(1, 1), padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project2)
project2 = BatchNormalization()(project2)
project2 = Add()([project1, project2])

########### Mobilenet block bneck 5x5 (128 --> 128) #################
expand3 = Conv2D(576, kernel_size=1, strides=1, kernel_regularizer=l2(1e-5), use_bias=False)(project2)
expand3 = BatchNormalization()(expand3)
expand3 = HardSwish()(expand3)
dw3 = DepthwiseConv2D(kernel_size=(5,5), strides=(1,1), padding='same', depthwise_regularizer=l2(1e-5), use_bias=False)(expand3)
dw3 = BatchNormalization()(dw3)
se_gap3 = GlobalAveragePooling2D()(dw3)
se_gap3 = Reshape([1, 1, -1])(se_gap3)
se3 = Conv2D(144, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se_gap3)
se3 = Activation('relu')(se3)
se3 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(se3)
se3 = HardSigmoid()(se3)
se3 = Multiply()([expand3, se3])
project3 = HardSwish()(se3)
project3 = Conv2D(128, kernel_size=(1, 1), padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project3)
project3 = BatchNormalization()(project3)
project3 = Add()([project2, project3])


########## Classification ##########
x2 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(project3)
x2 = BatchNormalization()(x2)
x2 = HardSwish()(x2)
x2 = GlobalAveragePooling2D()(x2)
x2 = Reshape([1, 1, -1])(x2)


######### Image Attention Model #########
### Block 1 ###
x3 = SeparableConv2D(32, kernel_size=(3, 3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), pointwise_regularizer=l2(1e-5), use_bias=False)(im_in)
x3 = BatchNormalization()(x3)
x3 = HardSwish()(x3)
x3 = Attention(32)(x3)

### Block 2 ###
x4 = SeparableConv2D(64, kernel_size=(3, 3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), pointwise_regularizer=l2(1e-5), use_bias=False)(x3)
x4 = BatchNormalization()(x4)
x4 = HardSwish()(x4)
x4 = Attention(64)(x4)

### Block 3 ###
x5 = SeparableConv2D(128, kernel_size=(3, 3), strides=(2,2), padding='same', depthwise_regularizer=l2(1e-5), pointwise_regularizer=l2(1e-5), use_bias=False)(x4)
x5 = BatchNormalization()(x5)
x5 = HardSwish()(x5)
x5 = Attention(128)(x5)

### final stage ###
x6 = Conv2D(576, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5), use_bias=False)(x5)
x6 = BatchNormalization()(x6)
x6 = HardSwish()(x6)
x6 = GlobalAveragePooling2D()(x6)
x6 = Reshape([1, 1, -1])(x6)

######## final addition #########

x2 = Add()([x2, x6])
x2 = Conv2D(1020, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5))(x2)
x2 = HardSwish()(x2)
x2 = Dropout(0.2)(x2)
x2 = Conv2D(2, kernel_size=1, strides=1, padding='valid', kernel_regularizer=l2(1e-5))(x2)
x2 = Activation('softmax')(x2)
x2 = Lambda(lambda x: K.squeeze(x, 1))(x2)
x2 = Lambda(lambda x: K.squeeze(x, 1))(x2)

model_top = Model(inputs=im_in, outputs=x2)
model_top.summary()

(1, 1, 32, 4)
(1, 1, 64, 8)
(1, 1, 128, 16)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 3, 3, 512)    722496      input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3, 3, 576)    294912      model_1[1][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3, 3, 576)    2304        conv2d_1[0][0]                   
________________________________________________

In [17]:
# optimizer = SGD(lr=1e-3, momentum=0.9, nesterov=True)
optimizer = Adam()
model_top.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
callback_list = [EarlyStopping(monitor='val_acc', patience=100), 
                 ReduceLROnPlateau(monitor='loss', factor=np.sqrt(0.5), cooldown=0, patience=5, min_lr=0.5e-5)]
output = model_top.fit_generator(ft_gen, steps_per_epoch=200, epochs=300,
                                  validation_data=validation_generator, validation_steps=len(validation_generator), callbacks=callback_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/300
200/200 [==============================] - 86s 429ms/step - loss: 0.7025 - acc: 0.6183 - val_loss: 0.9523 - val_acc: 0.5000
Epoch 2/300
200/200 [==============================] - 76s 381ms/step - loss: 0.5348 - acc: 0.7343 - val_loss: 0.9417 - val_acc: 0.5105
Epoch 3/300
200/200 [==============================] - 83s 416ms/step - loss: 0.4535 - acc: 0.7805 - val_loss: 5.1508 - val_acc: 0.5438
Epoch 4/300
200/200 [==============================] - 82s 410ms/step - loss: 0.3932 - acc: 0.8190 - val_loss: 3.1349 - val_acc: 0.6293
Epoch 5/300
200/200 [==============================] - 96s 479ms/step - loss: 0.3602 - acc: 0.8379 - val_loss: 0.0110 - val_acc: 0.5000
Epoch 6/300
200/200 [==============================] - 100s 501ms/step - loss: 0.3090 - acc: 0.8660 - val_loss: 4.7389 - val_acc: 0.5270
Epoch 7/300
200/200 [==============================] - 78s 392ms/step - loss: 0.2887 - ac

200/200 [==============================] - 90s 451ms/step - loss: 0.0338 - acc: 0.9934 - val_loss: 3.9116 - val_acc: 0.6301
Epoch 59/300
200/200 [==============================] - 93s 465ms/step - loss: 0.0313 - acc: 0.9937 - val_loss: 2.0788 - val_acc: 0.7847
Epoch 60/300
200/200 [==============================] - 87s 434ms/step - loss: 0.0359 - acc: 0.9921 - val_loss: 16.1306 - val_acc: 0.5000
Epoch 61/300
200/200 [==============================] - 79s 393ms/step - loss: 0.0379 - acc: 0.9912 - val_loss: 2.0784 - val_acc: 0.8246
Epoch 62/300
200/200 [==============================] - 78s 389ms/step - loss: 0.0323 - acc: 0.9935 - val_loss: 1.6274 - val_acc: 0.8202
Epoch 63/300
200/200 [==============================] - 78s 390ms/step - loss: 0.0326 - acc: 0.9934 - val_loss: 5.7599 - val_acc: 0.6352
Epoch 64/300
200/200 [==============================] - 78s 392ms/step - loss: 0.0316 - acc: 0.9932 - val_loss: 3.3653 - val_acc: 0.7338
Epoch 65/300
200/200 [==============================]

Epoch 118/300
200/200 [==============================] - 68s 338ms/step - loss: 0.0150 - acc: 0.9982 - val_loss: 2.0180 - val_acc: 0.7844
Epoch 119/300
200/200 [==============================] - 68s 341ms/step - loss: 0.0131 - acc: 0.9990 - val_loss: 0.4235 - val_acc: 0.9019
Epoch 120/300
200/200 [==============================] - 70s 351ms/step - loss: 0.0138 - acc: 0.9987 - val_loss: 2.7249 - val_acc: 0.7396
Epoch 121/300
200/200 [==============================] - 68s 339ms/step - loss: 0.0134 - acc: 0.9988 - val_loss: 0.9185 - val_acc: 0.8846
Epoch 122/300
200/200 [==============================] - 71s 353ms/step - loss: 0.0143 - acc: 0.9985 - val_loss: 1.5241 - val_acc: 0.8359
Epoch 123/300
200/200 [==============================] - 67s 333ms/step - loss: 0.0150 - acc: 0.9982 - val_loss: 0.4256 - val_acc: 0.9116
Epoch 124/300
200/200 [==============================] - 68s 341ms/step - loss: 0.0150 - acc: 0.9982 - val_loss: 3.3411 - val_acc: 0.6925
Epoch 125/300
200/200 [===========

200/200 [==============================] - 63s 314ms/step - loss: 0.0107 - acc: 0.9995 - val_loss: 0.3664 - val_acc: 0.9220
Epoch 178/300
200/200 [==============================] - 63s 315ms/step - loss: 0.0125 - acc: 0.9989 - val_loss: 0.4944 - val_acc: 0.9200
Epoch 179/300
200/200 [==============================] - 64s 319ms/step - loss: 0.0118 - acc: 0.9988 - val_loss: 0.4015 - val_acc: 0.9179
Epoch 180/300
200/200 [==============================] - 63s 314ms/step - loss: 0.0124 - acc: 0.9990 - val_loss: 0.3566 - val_acc: 0.9223
Epoch 181/300
200/200 [==============================] - 62s 310ms/step - loss: 0.0115 - acc: 0.9992 - val_loss: 0.3637 - val_acc: 0.9187
Epoch 182/300
200/200 [==============================] - 63s 313ms/step - loss: 0.0120 - acc: 0.9991 - val_loss: 0.3892 - val_acc: 0.9103
Epoch 183/300
200/200 [==============================] - 62s 310ms/step - loss: 0.0119 - acc: 0.9989 - val_loss: 0.3896 - val_acc: 0.9083
Epoch 184/300
200/200 [=========================

200/200 [==============================] - 70s 352ms/step - loss: 0.0113 - acc: 0.9995 - val_loss: 0.3303 - val_acc: 0.9202
Epoch 237/300
200/200 [==============================] - 62s 312ms/step - loss: 0.0118 - acc: 0.9991 - val_loss: 0.3549 - val_acc: 0.9169
Epoch 238/300
200/200 [==============================] - 69s 344ms/step - loss: 0.0115 - acc: 0.9993 - val_loss: 0.3992 - val_acc: 0.9162
Epoch 239/300
200/200 [==============================] - 63s 315ms/step - loss: 0.0117 - acc: 0.9990 - val_loss: 0.4196 - val_acc: 0.9081
Epoch 240/300
200/200 [==============================] - 69s 346ms/step - loss: 0.0109 - acc: 0.9993 - val_loss: 0.3320 - val_acc: 0.9192
Epoch 241/300
200/200 [==============================] - 62s 311ms/step - loss: 0.0124 - acc: 0.9987 - val_loss: 0.3483 - val_acc: 0.9134
Epoch 242/300
200/200 [==============================] - 79s 394ms/step - loss: 0.0115 - acc: 0.9991 - val_loss: 0.3212 - val_acc: 0.9289
Epoch 243/300
200/200 [=========================

200/200 [==============================] - 62s 308ms/step - loss: 0.0110 - acc: 0.9994 - val_loss: 0.4009 - val_acc: 0.9173
Epoch 296/300
200/200 [==============================] - 62s 312ms/step - loss: 0.0118 - acc: 0.9990 - val_loss: 0.4172 - val_acc: 0.9155
Epoch 297/300
200/200 [==============================] - 62s 308ms/step - loss: 0.0114 - acc: 0.9989 - val_loss: 0.3636 - val_acc: 0.9199
Epoch 298/300
200/200 [==============================] - 63s 313ms/step - loss: 0.0111 - acc: 0.9992 - val_loss: 0.3681 - val_acc: 0.9229
Epoch 299/300
200/200 [==============================] - 62s 311ms/step - loss: 0.0123 - acc: 0.9988 - val_loss: 0.4922 - val_acc: 0.9197
Epoch 300/300
200/200 [==============================] - 63s 317ms/step - loss: 0.0112 - acc: 0.9991 - val_loss: 0.3136 - val_acc: 0.9259


In [18]:
output_score50 = []
output_class50 = []
answer_class50 = []
answer_class50_1 =[]

for i in trange(len(test50_generator)):
    output50 = model_top.predict_on_batch(test50_generator[i][0])
    output_score50.append(output50)
    answer_class50.append(test50_generator[i][1])
    
output_score50 = np.concatenate(output_score50)
answer_class50 = np.concatenate(answer_class50)

output_class50 = np.argmax(output_score50, axis=1)
answer_class50_1 = np.argmax(answer_class50, axis=1)

print(output_class50)
print(answer_class50_1)

100%|██████████| 294/294 [00:53<00:00,  5.47it/s]

[0 0 0 ... 0 1 1]
[0 0 0 ... 1 1 1]


In [19]:
cm50 = confusion_matrix(answer_class50_1, output_class50)
report50 = classification_report(answer_class50_1, output_class50)

recall50 = cm50[0][0] / (cm50[0][0] + cm50[0][1])
fallout50 = cm50[1][0] / (cm50[1][0] + cm50[1][1])

fpr50, tpr50, thresholds50 = roc_curve(answer_class50_1, output_score50[:, 1], pos_label=1.)
eer50 = brentq(lambda x : 1. - x - interp1d(fpr50, tpr50)(x), 0., 1.)
thresh50 = interp1d(fpr50, thresholds50)(eer50)

print(report50)
print(cm50)
print("AUROC: %f" %(roc_auc_score(answer_class50_1, output_score50[:, 1])))
print(thresh50)
print('test_acc: ', len(output_class50[np.equal(output_class50, answer_class50_1)]) / len(output_class50))

              precision    recall  f1-score   support

           0       0.90      0.87      0.89     18788
           1       0.88      0.90      0.89     18778

    accuracy                           0.89     37566
   macro avg       0.89      0.89      0.89     37566
weighted avg       0.89      0.89      0.89     37566

[[16409  2379]
 [ 1792 16986]]
AUROC: 0.952616
0.7261521220224199
test_acc:  0.8889687483362615
